In [ ]:
from utils.config import *
import pandas as pd
import numpy as np
from utils import dataset as ds
from utils import emg_handler
import os

tmp = ds.movload(os.path.join(DATA_PATH, 'behavioural', 'efcp_100_01.mov'))

In [21]:
file = pd.read_csv(os.path.join(DATA_PATH,'emg','subj100','Trial_1.csv'), header=None, delimiter=',', skiprows=5) 


/var/folders/5y/1pn48fgn7yb77gc17h_5jms80000gp/T/ipykernel_23913/294276833.py:1: DtypeWarning: Columns (0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31) have mixed types. Specify dtype option on import or set low_memory=False.
  file = pd.read_csv(os.path.join(DATA_PATH,'emg','subj100','Trial_1.csv'), header=None, delimiter=',', skiprows=5)


In [ ]:
col_names = ['Analog 1', 'ext_D3', 'ext_D4', 'ext_D5', 'ext_D1', 'ext_D2', 'flx_D3', 'flx_D4', 'flx_D2', 'flx_D1', 'flx_D5']
channel_names = ['ext_D1', 'ext_D2', 'ext_D3', 'ext_D4', 'ext_D5', 'flx_D1', 'flx_D2', 'flx_D3', 'flx_D4', 'flx_D5']

raw_emg = pd.DataFrame()

# find the trial indices from the trigger channel:

# for name in channel_names:
#     for i, col_name in enumerate(x.iloc[0].values):
#         if name in col_name:
#             print(f'{x.iloc[0].values[i]}')

Analog 1 Time Series (s)
 Analog 1 (V)
 ext_D3 Time Series (s)
 ext_D3 (mV)
 ext_D4 Time Series (s)
 ext_D4 (mV)
 ext_D5 Time Series (s)
 ext_D5 (mV)
 ext_D1 Time Series (s)
 ext_D1 (mV)
 ext_D2 Time Series (s)
 ext_D2 (mV)
 flx_D3 Time Series (s)
 flx_D3 (mV)
 flx_D4 Time Series (s)
 flx_D4 (mV)
 flx_D2 Time Series (s)
 flx_D2 (mV)
 flx_D1 Time Series (s)
 flx_D1 (mV)
 flx_D5 Time Series (s)
 flx_D5 (mV)
